In [59]:
# Matplotlib
import requests
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Torch
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import pandas as pd

In [60]:
# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [61]:
taxi_availability_file_path = "taxi_availability.csv"

taxi_df = pd.read_csv(taxi_availability_file_path, delimiter=",")
# Save this just in case we need it in the future.
taxi_df_coordinates = taxi_df["Coordinates[]"]
taxt_df_datetime = taxi_df["DateTime"]
taxi_df = taxi_df.drop(columns = "Coordinates[]")

taxi_df["DateTime"] = pd.to_datetime(taxi_df["DateTime"])


taxi_df["IsWeekend"] = (taxi_df["DateTime"].dt.weekday >= 5).astype(int)
taxi_df["Hour"] = taxi_df["DateTime"].dt.hour + 1  # Convert 0-23 to 1-24
taxi_df = taxi_df.drop(columns = "DateTime")

#it takes 23:59:59 as midnight, same for every DateTime value.
print(taxi_df.iloc[0])


#---------------Normalise-----------------------
# Drop 'DateTime' as it's no longer needed

# Normalize the 'Hour' and 'IsWeekend' columns (if needed)
taxi_df = taxi_df[:5120]
print(taxi_df.shape)
scaler = MinMaxScaler()
taxi_df[["Hour", "IsWeekend"]] = scaler.fit_transform(taxi_df[["Hour", "IsWeekend"]])
taxi_df = taxi_df.apply(pd.to_numeric, errors='coerce')
taxi_df_output  = taxi_df["Taxi Available in Selected Box Area"]
taxi_df = taxi_df.drop(columns = "Taxi Available in Selected Box Area")

# Convert to NumPy arrays
input_data = taxi_df.values  # Shape: (5120, num_features)
output_data = taxi_df_output.values  # Shape: (5120,)

# Define sequence length
seq_length = 24

# Function to create sequences
def create_sequences(data, labels, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i : i + seq_length])  # Input sequence
        ys.append(labels[i + seq_length])   # Corresponding target
    return np.array(xs), np.array(ys)

# Create sequences
X, y = create_sequences(input_data, output_data, seq_length)

print("X[0]: ", X[0])
print("y[0]: ", y)

Taxi Available throughout SG           1924
Taxi Available in Selected Box Area      79
IsWeekend                                 0
Hour                                     24
Name: 0, dtype: object
(5120, 4)
X[0]:  [[1.92400000e+03 0.00000000e+00 1.00000000e+00]
 [2.25900000e+03 0.00000000e+00 9.56521739e-01]
 [2.40000000e+03 0.00000000e+00 9.13043478e-01]
 [2.67700000e+03 0.00000000e+00 8.69565217e-01]
 [2.43700000e+03 0.00000000e+00 8.26086957e-01]
 [1.47900000e+03 0.00000000e+00 7.82608696e-01]
 [1.76500000e+03 0.00000000e+00 7.39130435e-01]
 [1.87700000e+03 0.00000000e+00 6.95652174e-01]
 [1.90800000e+03 0.00000000e+00 6.52173913e-01]
 [2.04600000e+03 0.00000000e+00 6.08695652e-01]
 [1.57600000e+03 0.00000000e+00 5.65217391e-01]
 [2.00000000e+03 0.00000000e+00 5.21739130e-01]
 [2.27000000e+03 0.00000000e+00 4.78260870e-01]
 [2.50000000e+03 0.00000000e+00 4.34782609e-01]
 [2.40600000e+03 0.00000000e+00 3.91304348e-01]
 [2.42800000e+03 0.00000000e+00 3.47826087e-01]
 [1.82600000e+03

In [62]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

trainX = torch.tensor(X, dtype=torch.float32)
trainY = torch.tensor(y, dtype=torch.float32)
print(trainX.shape)
print(trainY.shape)

torch.Size([5096, 24, 3])
torch.Size([5096])


In [63]:
t = np.linspace(0, 100, 1000)
data = np.sin(t)

def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(data, seq_length)

trainX = torch.tensor(X[:, :, None], dtype=torch.float32)
trainY = torch.tensor(y[:, None], dtype=torch.float32)

In [64]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out, hn, cn

In [67]:
model = LSTMModel(input_dim=1, hidden_dim=100, layer_dim=1, output_dim=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [68]:
num_epochs = 100
h0, c0 = None, None

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    outputs, h0, c0 = model(trainX, h0, c0)

    loss = criterion(outputs, trainY)


    loss.backward()
    optimizer.step()

    h0 = h0.detach()
    c0 = c0.detach()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.2877
Epoch [20/100], Loss: 0.1027
Epoch [30/100], Loss: 0.0419
Epoch [40/100], Loss: 0.0167
Epoch [50/100], Loss: 0.0032
Epoch [60/100], Loss: 0.0007
Epoch [70/100], Loss: 0.0004
Epoch [80/100], Loss: 0.0004
Epoch [90/100], Loss: 0.0001
Epoch [100/100], Loss: 0.0001


In [69]:
print(outputs)


tensor([[ 0.8432],
        [ 0.8907],
        [ 0.9297],
        [ 0.9600],
        [ 0.9815],
        [ 0.9938],
        [ 0.9969],
        [ 0.9908],
        [ 0.9752],
        [ 0.9502],
        [ 0.9158],
        [ 0.8722],
        [ 0.8195],
        [ 0.7580],
        [ 0.6882],
        [ 0.6107],
        [ 0.5262],
        [ 0.4358],
        [ 0.3404],
        [ 0.2412],
        [ 0.1396],
        [ 0.0369],
        [-0.0655],
        [-0.1666],
        [-0.2649],
        [-0.3597],
        [-0.4499],
        [-0.5348],
        [-0.6137],
        [-0.6863],
        [-0.7518],
        [-0.8100],
        [-0.8605],
        [-0.9029],
        [-0.9370],
        [-0.9624],
        [-0.9789],
        [-0.9864],
        [-0.9846],
        [-0.9735],
        [-0.9531],
        [-0.9233],
        [-0.8843],
        [-0.8361],
        [-0.7791],
        [-0.7137],
        [-0.6402],
        [-0.5594],
        [-0.4720],
        [-0.3789],
        [-0.2813],
        [-0.1804],
        [-0.

In [ ]:
print(trainY)

tensor([ 77.,  97.,  88.,  ..., 112., 111., 140.])
